In [ ]:
import numpy as np
import pandas as pd
import lightkurve as lk
import matplotlib.pyplot as plt
import re
import os
from astropy.io import fits

import warnings

warnings.filterwarnings("ignore")

In [9]:
path_data = '../results/TPF_dataframe.csv'
df = pd.read_csv(path_data)

folder = '../results/TPF_data/'
subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]

In [10]:
for index, row in df.iterrows():
    # Check if TPF was found for this star
    if row['TPF_Found']:
        # Create a folder for the star
        name = row['Star_Name']
        
        #join name with underscores
        name = name.replace(" ", "_")
        star_folder = f"{folder}{name}"

        if star_folder in subfolders:
            print(f"Folder for {name} already exists. Skipping...")
            continue

        os.makedirs(star_folder, exist_ok=True)
        # Search for the TPF
        found_name = row['Found_Star_Name']
        tpf = lk.search_targetpixelfile(row['Found_Star_Name'])
        n_tpf = len(tpf)

        print(f'{name} ({found_name})', end=': ')
        print(f"Found {n_tpf} TPFs")
        # Loop through the search result
        for i in range(n_tpf):
            print(f"Downloading TPF {i+1}/{n_tpf}", end='\r')
            # Get the exposure time
            exptime = tpf[i].exptime
            match = re.search(r'\d+', str(exptime))
            number = int(match.group())
            # Create a folder for this exposure time within the star's folder
            exptime_folder = f"{star_folder}/exp_{number}"
            os.makedirs(exptime_folder, exist_ok=True)

            try:
                tpf_file = tpf[i].download()
                fits_hdu = tpf_file.to_lightcurve(aperture_mask=tpf_file.pipeline_mask).to_fits()
                header = fits_hdu[0].header
                telescope, date, object = header['TELESCOP'], header['DATE'], header['OBJECT']
                path = f"{exptime_folder}/{name}_{telescope}_{date}_{object}_{i}.fits"
                fits_hdu.writeto(path, overwrite=True)
                
            except lk.LightkurveError as e:
                print(f"Error downloading TPF for {row['Star_Name']}: {e}")
            except FileNotFoundError as e:
                print(f"Error downloading TPF for {row['Star_Name']}: {e}")
            except IndexError as e:
                print(f"IndexError: {e}. Skipping iteration {i}.")

Folder for Kepler-1663 already exists. Skipping...
Folder for K2-33 already exists. Skipping...
Folder for TOI-1227 already exists. Skipping...
Folder for HD_114082 already exists. Skipping...
Folder for HIP_67522 already exists. Skipping...
Folder for WASP-25 already exists. Skipping...
Folder for AU_Mic already exists. Skipping...
Folder for V1298_Tau already exists. Skipping...
Folder for HD_109833 already exists. Skipping...
Folder for CoRoT-18 already exists. Skipping...
Folder for TOI-837 already exists. Skipping...
Folder for KOI-7913_A already exists. Skipping...
Folder for KOI-7368 already exists. Skipping...
Folder for DS_Tuc_A already exists. Skipping...
Folder for Kepler-1643 already exists. Skipping...
Folder for TOI-942 already exists. Skipping...
Folder for CoRoT-20 already exists. Skipping...
Folder for Kepler-1928 already exists. Skipping...
Folder for Kepler-970 already exists. Skipping...
Folder for K2-284 already exists. Skipping...
Folder for TOI-251 already exists